# Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup and Imports

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [36]:
# For example: 
import os
from dotenv import load_dotenv                  # For loading environment variables from .env file
from datetime import datetime                   # For websearch metadata assignment

from typing import List, Dict, Any, Annotated
from pydantic import BaseModel, Field

from lib.agents import Agent                    # For Agent class
from lib.llm import LLM                         # For Large Language Model
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool                    # For defining tools    
from lib.parsers import PydanticOutputParser    # To return structured output from LLM

from chromadb import PersistentClient           # For Vector Database
# import chromadb

from tavily import TavilyClient                 # For Web Search


In [26]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
VOCAREUM_OPENAI_API_KEY=os.getenv("VOCAREUM_OPENAI_API_KEY")
VOCAREUM_BASE_URL=os.getenv("VOCAREUM_BASE_URL")

Setting up ChromaDB paths and names. Should be consistent with what you used in the `Udaplay_01.ipynb` file.

In [9]:
CHROMADB_PATH = "chromadb"
CHROMADB_COLLECTION_NAME = "udaplay"

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [ ]:
def retrieve_game(query: str, n_results: int=4) -> List:
    """
#    Semantic search: Finds most relevant results in the vector DB
#    args:
#    - query: a question about game industry. 
#    - n_results: number of results to return (default is 4)
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
    """

    # collection.query(
        # query_texts=[query],
        # n_results=n_results,
        # include=['metadatas', 'documents', 'distances'] # TODO: look at collection.peek() to understand this better.
    # )

    # These have been created in Phase 1
    chroma_client = PersistentClient(path=CHROMADB_PATH)
    collection = chroma_client.get_collection(CHROMADB_COLLECTION_NAME)

    results = collection.query(
        query_texts=[query], 
        n_results=5
        )
    
    return results["documents"][0]

In [28]:
# To test the function
query = "Tell me about Gran Turismo"
# response = retrieve_game(query)
# print(response)

#### Evaluate Retrieval Tool

In [33]:
class EvaluationReport(BaseModel):
    useful: bool = Field(..., description="Indicates if the retrieved documents are useful for answering the question")
    description: str = Field(..., description="Detailed explanation of the evaluation result, including why the documents are or are not useful")


def evaluate_retrieval(question: str, retrieved_docs: list) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents, this function 
    will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    Outputs:
    EvaluationReport:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    system_prompt = """
        You are an expert in evaluating the usability of documents for answering user queries.
        Your task is to evaluate if the provided sources are enough to respond the user query.
        Give a detailed explanation, so it's possible to take an action to accept it or not.
    """

    user_prompt = f"""
        User query:\n{question}\n
        Retrieved documents:\n{retrieved_docs}
    """

    messages = [
        SystemMessage(content=system_prompt),
        UserMessage(content=user_prompt)
    ]

    llm_judge = LLM(
        model="gpt-4.1-nano",
        temperature=0.0,
        api_key=VOCAREUM_OPENAI_API_KEY,
        base_url=VOCAREUM_BASE_URL
    )

    ai_message = llm_judge.invoke(input=messages, response_format=EvaluationReport)
    parser = PydanticOutputParser(model_class=EvaluationReport)
    return parser.parse(ai_message)


In [47]:
# To test the function
response_valid = evaluate_retrieval(query, response)
print(response_valid.model_dump_json(indent=2))

{
  "useful": true,
  "description": "The retrieved documents include detailed information about the Gran Turismo series, specifically mentioning the original game and its sequel, Gran Turismo 5. They describe the games as realistic racing simulators with a wide selection of vehicles and tracks, and highlight their significance in the racing genre. Although some documents mention other racing games, the core information about Gran Turismo is present, making these sources sufficient to answer the user query about the series."
}


#### Game Web Search Tool

In [ ]:
class MetaData(BaseModel):
    timestamp: str = Field(..., description="The time the web search was performed, in ISO format (YYYY-MM-DD HH:MM:SS.mmmmmm)")
    query: str = Field(..., description="The original search query")

class WebSearchResult(BaseModel):
    answer: str = Field(..., description="Answer to the search query")
    results: List[Dict] = Field(..., description="List of search results")
    search_metadata: MetaData  = Field(..., description="Metadata of the web search")

# Modified from Course module 06 solution notebook (Web Search with Tavily)
def game_web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API.
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    Output: 
    WebSearchResult:
    - answer (str): Direct answer to the query, if available
    - results (List[Dict]): List of search results, each containing:
    - search_metadata (Dict): Metadata about the search, including timestamp and query
    """
    client = TavilyClient(api_key=TAVILY_API_KEY)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    # metadata_parser = PydanticOutputParser(model_class=WebSearchResult)
    # validated_results = metadata_parser.parse(ai_message=AIMessage(content=str(formatted_results)))
    validated_results = WebSearchResult(**formatted_results)

    return validated_results

In [48]:
# To test the function
web_search_response = game_web_search(query)
print(web_search_response.model_dump_json(indent=2))

{
  "answer": "Gran Turismo is a racing simulation video game series developed by Polyphony Digital and published by Sony. It is known for its realistic driving physics and detailed vehicle models. The series has launched eight mainline titles and sold over 90 million units worldwide.",
  "results": [
    {
      "url": "https://playstation-studios.fandom.com/wiki/Gran_Turismo_(series)",
      "title": "Gran Turismo (series) | PlayStation Studios Wiki - Fandom",
      "content": "Gran Turismo, also known as GT, is a series of racing simulation video games created and developed by Polyphony Digital, and published by Sony Interactive Entertainment for the PlayStation brand of consoles. Originally released for the very first PlayStation \"PlayStation (console)\"), Gran Turismo is among Sony's oldest franchises and its longest-running franchise, having launched in 1997. [...] The Gran Turismo series received significant praise since its inception, and has been credited with establishing Po

### Agent

In [ ]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

In [ ]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes